In [10]:
import re, copy

#Maximal size of 32 bit number
maxSize = 4294967296

class MyException(Exception): pass

#Initialization of new BigInt number
#Dividing input hex number by parts with 8 digits, turn them into int and reverse because of little endian
class BIG_INTEGER:
    def __init__(self, hexa = ''): 
        self.hexa = hexa
        
    def bigint(self): 
        return [int(self.hexa[max(x - 8, 0):x], 16) for x in range(len(self.hexa), 0, -8)]
    
#Taking little endian BigInt and turn into hex 
def BIG_INT_TO_HEX(num):
    num.reverse()
    res, pattern = '', r'^0+'
    for i in range (len(num)):
        temp = hex(num[i]).split('x')[-1]
        if len(temp) < 8: temp = ''.join(list((8 - len(temp))*'0') + list(temp))
        res += temp
    return re.sub(pattern, '',res)
    
#Addition of two BigInt numbers
def LongAdd(num1,num2):
    c, carry = [], 0
    for i in range(len(num1)) if len(num1)>=len(num2) else range(len(num2)):
        temp1 = num1[i] if i < len(num1) else 0 
        temp2 = num2[i] if i < len(num2) else 0
        temp = temp1 + temp2 + carry
        c.insert(i, temp) if temp <= maxSize else c.insert(i, (temp - maxSize)) #cast to ffffffff
        carry = temp >> 32 #Carry bit
    if carry == 1: c.append(1) 
    return c 

#Substraction 
def LongSub(num1, num2):
    c, borrow = [], 0
    for i in range(len(num1)) if len(num1) >= len(num2) else range(len(num2)):
        temp1 = num1[i] if i < len(num1) else 0 
        temp2 = num2[i] if i < len(num2) else 0    
        temp = temp1 - temp2 - borrow
        if temp >= 0:
            c.insert(i, temp)
            borrow = 0
        else:
            c.insert(i, temp + maxSize)
            borrow = 1
    if borrow == 1:raise MyException('num1 < num2')
    return c

#Comparsion
def LongCmp(num1, num2):
    if len(num1) > len(num2):temp = 1
    elif len(num1) == len(num2):
        for i in range(len(num1)):
            if num1[i] > num2[i]:temp = 1
            elif num1[i] < num2[i]:temp = -1
            elif num1 == num2:temp = 0
    else:temp = -1
    return temp
    
#Multiplication of BigInt number with one digit of BigInt number
def LongMulOneDigit(num1, num2):
    carry, c = 0, []
    for i in range(len(num1)):
        temp = num1[i]*num2 + carry
        c.insert(i, temp) if temp <= maxSize else c.insert(i, temp % (maxSize))
        carry = temp // (maxSize)
    if carry > 0: c.append(carry)
    return c 

#Multiplication of big number and 2^32^n (Just put 0 in the end)
def LongShiftDigitsToHigh(num, n):
    numb = copy.deepcopy(num)
    for i in range(n):numb.insert(0,0)
    return numb

#Multiplication of two big numbers
def LongMul(num1, num2):
    c = []
    for i in range(len(num1)) if len(num1)>=len(num2) else range(len(num2)):
        if len(num1)>=len(num2):
            temp = LongMulOneDigit(num2, num1[i])
        else:temp = LongMulOneDigit(num1, num2[i])
        temp = LongShiftDigitsToHigh(temp, i)
        c = LongAdd(c, temp)        
    return c

#Multiplication of BigInt and 2^n
def LongShiftBitsToHigh(num, n):
    nums = copy.deepcopy(num)
    hex_string = int('0x' + BIG_INT_TO_HEX(nums), base=16)
    if n > 0: hex_string = hex_string << n
    return BIG_INTEGER(hex(hex_string)[2:]).bigint()

#Length of BigInt in bits
def BitLength(num):
    i = len(num) - 1
    while i > 0 and num[i] == 0: i -= 1
    return i * 32 + num[i].bit_length()

#Division of big number and 2^32^n
def LongShiftBitsToLow(a, n):
    ints, bits = n // 32, n % 32

    if bits == 0: return LongShiftDigitsToHigh(a, ints)

    hi_bits = a[-1] >> (32 - bits)
    c = [0] * (len(a) + ints + (1 if hi_bits != 0 else 0))

    i = len(c)-1
    j = len(a)-1
    if hi_bits != 0:
        c[len(c)-1] = hi_bits
        i -= 1
    
    while j > 0:
        t1 = (a[j] << bits) & (2**32-1)
        t2 = a[j-1] >> (32 - bits)
        c[i] = t1 | t2
        i -= 1
        j -= 1
    c[i] = (a[j] << bits) & (2**32-1)

    return c

#Division of  two BigIntegers 
def LongDiv(num1, num2):
    k = BitLength(num2)
    r = num1.copy()
    q = []
    temp = [1]

    while LongCmp(r, num2) >= 0:
        t = BitLength(r)
        c = LongShiftBitsToLow(num2, t - k)
        if LongCmp(r, c) < 0:
            t -= 1
            c = LongShiftBitsToLow(num2, t - k)
        r = LongSub(r, c)
        q = LongAdd(q, LongShiftBitsToLow(temp, t - k))
    return q, r

#Recognize the value of bit on "index" position
def BitValueForIndex(num, index):
    l, m = index % 32, index // 32
    return (num[m] >> l) & 1

#BiInt to the power of BigInt
def LongPow(num1, num2):
    c = [1]
    for i in range(BitLength(num2)):
        if(BitValueForIndex(num2, i) == 1): c = LongMul(c, num1)
        num1 = LongMul(num1, num1)
    return c
    
num1 = BIG_INTEGER(input('Input hex number: \n')).bigint()
num2 = BIG_INTEGER(input('Input hex number: \n')).bigint()

print('\nNumber 1 int array: ', num1, '\n')
print('Number 2 int array: ', num2, '\n')

print('Addiction: ', BIG_INT_TO_HEX(LongAdd(num1,num2)), '\n')

print('Substraction: ', BIG_INT_TO_HEX(LongSub(num1,num2)), '\n')

print('Comparsion: ', (LongCmp(num1,num2)), '\n')

print('Multipliplication on 1 digit : ', BIG_INT_TO_HEX(LongMulOneDigit(num1,2)), '\n')

print('Multipliplication : ', BIG_INT_TO_HEX(LongMul(num1, num2)), '\n')

print('LongDiv : ', BIG_INT_TO_HEX(LongDiv(num1, num2)[0]), '\n')
print('LongDiv remainder : ', BIG_INT_TO_HEX(LongDiv(num1, num2)[1]), '\n')


print('LongPow by 3 : ', BIG_INT_TO_HEX(LongPow(num1, [3])), '\n')

################################################################




#Example of hex numbers with length 512 bit
#D4D2110984907B5625309D956521BAB4157B8B1ECE04043249A3D379AC112E5B9AF44E721E148D88A942744CF56A06B92D28A0DB950FE4CED2B41A0BD38BCE7D0BE1055CF5DE38F2A588C2C9A79A75011058C320A7B661C6CE1C36C7D870758307E5D2CF07D9B6E8D529779B6B2910DD17B6766A7EFEE215A98CAC300F2827DB
#3A7EF2554E8940FA9B93B2A5E822CC7BB262F4A14159E4318CAE3ABF5AEB1022EC6D01DEFAB48B528868679D649B445A753684C13F6C3ADBAB059D635A2882090FC166EA9F0AAACD16A062149E4A0952F7FAAB14A0E9D3CB0BE9200DBD3B0342496421826919148E617AF1DB66978B1FCD28F8408506B79979CCBCC7F7E5FDE7


Input hex number: 
D4D2110984907B5625309D956521BAB4157B8B1ECE04043249A3D379AC112E5B9AF44E721E148D88A942744CF56A06B92D28A0DB950FE4CED2B41A0BD38BCE7D0BE1055CF5DE38F2A588C2C9A79A75011058C320A7B661C6CE1C36C7D870758307E5D2CF07D9B6E8D529779B6B2910DD17B6766A7EFEE215A98CAC300F2827DB
Input hex number: 
3A7EF2554E8940FA9B93B2A5E822CC7BB262F4A14159E4318CAE3ABF5AEB1022EC6D01DEFAB48B528868679D649B445A753684C13F6C3ADBAB059D635A2882090FC166EA9F0AAACD16A062149E4A0952F7FAAB14A0E9D3CB0BE9200DBD3B0342496421826919148E617AF1DB66978B1FCD28F8408506B79979CCBCC7F7E5FDE7

Number 1 int array:  [254289883, 2844568624, 2130633237, 397833834, 1797853405, 3576264603, 131708648, 132502223, 3631248771, 3457955527, 2813747654, 274252576, 2811917569, 2777203401, 4124981490, 199296348, 3549154941, 3535018507, 2500846798, 757637339, 4117366457, 2839704652, 504663432, 2599702130, 2886807131, 1235473273, 3456369714, 360418078, 1696709300, 623943061, 2224061270, 3570536713] 

Number 2 int array:  [4159045095, 2043460807, 223

In [16]:
print('---------------------------------------Galua Field Operations---------------------------------------')
def initiationizealization(a, b, c, d, e, poly = 0):
    for i in (a, b, c, d, e): poly = poly | (1 << i)
    return poly


def mul(a, b, poly, c = 0, mod = 0): 
    for i in range(b.bit_length()):
        if b&(1 << i): 
            c = c ^ (a << i)
    while poly <= c:
        mod = c.bit_length() - poly.bit_length()
        div = poly << mod
        c = c ^ div
    return c

def a_pow_n(a, n, c = 1):
    temp = a
    for i in range(n.bit_length()):
        if n & (1 << i): c = mul(c, temp, poly_coef)
        temp = mul(temp, temp, poly_coef)
    return c

def revers(a, poly):
    m = poly.bit_length() - 1
    return a_pow_n(a, (1 << m) - 2)

a = 0x0117EBEDD8803B42EB0ED8F0AF834FC93E1DFF40FAF7287463ADED350AA25402EC1857F2
b = 0x00FA281D25C7441D215F19BC12B835422041470EE90793579DF2B5A3CDB8C8A4014CC3DE
n = 0x015F0784041656F6655A891BED41506E1DE6823541A1A05748691A676F7D06AEBB0F636E
poly_coef  =  initiationizealization(281, 9, 4, 1, 0)

print('a + b -', hex(a ^ b)[2:])
print('a * b -', hex(mul(a, b, poly_coef))[2:])
print('a * a -', hex(mul(a, a, poly_coef))[2:])
print('a ^ n -', hex(a_pow_n(a, n))[2:])
print('a ^-1 -', hex(revers(a, poly_coef))[2:])


---------------------------------------Galua Field Operations---------------------------------------
a + b - 1edc3f0fd477f5fca51c14cbd3b7a8b1e5cb84e13f0bb23fe5f5896c71a9ca6ed54942c
a * b - 19ed513fdca229f920c43d5240225cb2b4c5cdfb8a384b180aa4ec9b9b777250a06dca0
a * a - 1c971194c49f726a6201078cb1934c81b8788a2b793821584f53f4bd16b6db94e4e7396
a ^ n - 16802111e7993f1a5840ac7690f490e18e220e05a72d5d83263eeedf693fd58eb7f579c
a ^-1 - 12a67b598bb17810e0707b64fb27b5de06cf4a465c7e24ce7d9a76f1ffea68787b44e
